In [19]:
from qiskit.circuit.library import U1Gate
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile
import operator
from qiskit.visualization import plot_histogram


theta = 1 / 10
u = U1Gate(2 * np.pi * theta).control(1)

k = 10
qreg = QuantumRegister(2)
creg = ClassicalRegister(k)
qc = QuantumCircuit(qreg, creg)
# possible states for qubit at last index
# |1> and -|1> which are the eigenstates of U at eigenvalue e^{2*pi*i*theta}
# must be an eigenstate of Unitary matrix U
qc.x(1)

simulator = AerSimulator()
for i in range(k):
    qc.reset(0)
    qc.h(0)
    for j in range(k - i):
        qc.compose(u, [0, 1], inplace=True)
    if i != 0:
        for j in range(i):
            current_bit = int(theta_bits[j])
            qc.rx(-2 * np.pi * current_bit / 2 ** (i + 1 - j), 0).c_if(creg[j], 1)
    qc.h(0)
    qc.measure(0, i)
    isa_circuit = transpile(qc, simulator)
    counts = simulator.run(isa_circuit, shots=1024).result().get_counts()
    theta_bits = max(counts.items(), key=operator.itemgetter(1))[0]
# qc.draw(output="mpl")

In [20]:
# plot_histogram(counts, figsize=(20, 5))

In [21]:
# phase estimation
highest_probability_outcome = max(counts.items(), key=operator.itemgetter(1))[0][::-1]
print("Highest probability outcome:", highest_probability_outcome)
measured_theta = int(highest_probability_outcome, 2) / 2**k
print(
    "Using %d iterations with theta = %.3f, estimated theta = %.3f."
    % (k, theta, measured_theta)
)

Highest probability outcome: 0001111100
Using 10 iterations with theta = 0.100, estimated theta = 0.121.
